In [1]:
# Import libraries.
using Turing, StatsPlots, Random

In [50]:
# Declare our Turing model.
@model function agent(opponent_agent, my_action, opponent_action, opponent_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    if depth > 0 
        opp_action_chain = sample(opponent_agent(agent, opponent_action, my_action, depth-1, opponent_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=false);
        opp_alpha_1 = mean(opp_action_chain[:"alpha[1]"])
        opp_alpha_2 = mean(opp_action_chain[:"alpha[2]"])
        opp_alpha_3 = 1 - opp_alpha_1 - opp_alpha_2
        alpha ~ Dirichlet([opp_alpha_3, opp_alpha_1 , opp_alpha_2])
    else
        # Our prior belief about the probability of RPS.
        alpha ~ Dirichlet(ones(3)/3)
    end
    my_action ~ Categorical(vec(alpha))
    return my_action
end

agent (generic function with 6 methods)

In [56]:
# Declare our Turing model.
@model function agent(opponent_agent, my_history, opponent_history, opponent_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    for i in length(my_history)
        my_action = my_history[i]
        opponent_action = opponent_history[i]
        if depth > 0 
            opp_action_chain = sample(opponent_agent(agent, opponent_action, my_action, depth-1, opponent_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations, progress=false);
            opp_alpha_1 = mean(opp_action_chain[:"alpha[1]"])
            opp_alpha_2 = mean(opp_action_chain[:"alpha[2]"])
            opp_alpha_3 = 1 - opp_alpha_1 - opp_alpha_2
            alpha ~ Dirichlet([opp_alpha_3, opp_alpha_1 , opp_alpha_2])
        else
            # Our prior belief about the probability of RPS.
            alpha ~ Dirichlet(ones(3)/3)
        end
        my_action ~ Categorical(vec(alpha))
        return my_action
    end
end

agent (generic function with 6 methods)

In [57]:
my_action = 1
opponent_action = 1
opponent_depth = 1
my_depth = 1

1

In [58]:
chain = sample(agent(agent, [my_action], [opponent_action], opponent_depth, my_depth), PG(10), 5 , progress = false)
chain

Chains MCMC chain (5×6×1 Array{Float64,3}):

Log evidence      = 0.0
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3], my_action
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat 
      Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

    alpha[1]    0.3687    0.5008     0.2240   missing    1.3366   20.2795
    alpha[2]    0.3788    0.4656     0.2082   missing   10.3439    1.0714
    alpha[3]    0.2525    0.3829     0.1712   missing    4.3929    1.1982
   my_action    1.8000    0.8367     0.3742   missing    1.6667    2.2361

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0007    0.0031    0.0064    0.8849    0.9423
    alpha[2]    0.0071    0.0477    0.0760    0.7996    0.9513
    alpha[3]

In [59]:
function maximum_likelihood_action(list_of_pairs)
    max_key = -1
    max_value = -1
    for element in list_of_pairs
        key = element[1]
        value = element[2]
        if value > max_value
            max_value = value
            max_key = key
        end
    end
    return (max_key, max_value)
end

maximum_likelihood_action (generic function with 1 method)

In [51]:
function move(agent, other_agent, my_history, other_agent_history)
    opponent_action = length(other_agent_history) > 0 ? last(other_agent_history) : 1
    my_action = length(my_history) > 0 ? last(my_history) : 1
    chain = sample(agent(other_agent, my_history, other_agent_history, opponent_depth, my_depth), PG(1), 1, progress = false)
    alpha_1 = (1, mean(chain[:"alpha[1]"]))
    alpha_2 = (2, mean(chain[:"alpha[2]"]))
    alpha_3 = (3, 1 - alpha_1[2] - alpha_2[2])
    return maximum_likelihood_action([alpha_1, alpha_2, alpha_3])[1]
end

move (generic function with 2 methods)

In [52]:
move(agent, agent, [])

opponent_action=1 my_action=1

Chains MCMC chain (5×5×1 Array{Float64,3}):

Log evidence      = -2.868925665007797
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat 
      Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

    alpha[1]    0.4269    0.2904     0.1299   missing   10.6551    1.0691
    alpha[2]    0.1741    0.2231     0.0998   missing   46.7051    1.0146
    alpha[3]    0.3990    0.2169     0.0970   missing   11.5669    1.0632

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.1583    0.3283    0.3665    0.3839    0.8629
    alpha[2]    0.0245    0.0743    0.0893    0.1203    0.5231
    alpha[3]    0.0876    0.2927    0.5269    0.5514    0.5584


1

In [53]:
function game()
    first_player = agent
    second_player = agent
    num_of_simulations = 10
    first_player_history = []
    second_player_history = []
    for i in 1:num_of_simulations
        m1 = move(first_player, second_player, first_player_history, second_player_history)
        push!(first_player_history, m1)
        m2 = move(second_player, first_player, second_player_history, first_player_history)
        push!(second_player_history, m2)
        println("in simulation $i first player chose $m1 second player chose $m2")
    end
    return history
end

game (generic function with 1 method)

In [55]:
game()

opponent_action=1 my_action=1

Chains MCMC chain (5×5×1 Array{Float64,3}):

Log evidence      = -2.5846619879587664
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse       ess      rhat 
      Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

    alpha[1]    0.2601    0.2593     0.1160   missing   -4.4842    0.8780
    alpha[2]    0.4829    0.3114     0.1393   missing   -1.6260    0.7412
    alpha[3]    0.2570    0.2495     0.1116   missing    1.9920    1.7391

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0619    0.1390    0.1395    0.2647    0.6600
    alpha[2]    0.1359    0.2078    0.5226    0.7060    0.8357
    alpha[3]    0.0360    0.0965    0.1681    0.3379    0.6217


opponent_action=2 my_action=1

Chains MCMC chain (5×5×1 Array{Float64,3}):

Log evidence      = -2.368142158649826
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse            ess      rhat ⋯
      Symbol   Float64   Float64    Float64   Missing        Float64   Float64 ⋯

    alpha[1]    0.6341    0.3320     0.1485   missing        -2.6801    0.8172 ⋯
    alpha[2]    0.0654    0.1366     0.0611   missing   -136860.4588    1.0000 ⋯
    alpha[3]    0.3006    0.3354     0.1500   missing        19.9014    1.0353 ⋯

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.1432    0.5176    0.8187    0.8350    0.8912
    alpha[2]    0.0000    0.0000    0.0000    0.0174    0.2802
    alpha[3]    0.1087    0.1639    0.1650    0.1

in simulation 1 first player chose 2 second player chose 1
opponent_action=1 my_action=2

...

Chains MCMC chain (5×5×1 Array{Float64,3}):

Log evidence      = -2.5399168187837597
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Missing     Float64   Float64 

    alpha[1]    0.4499    0.3392     0.1517   missing      1.3766    5.6382
    alpha[2]    0.0275    0.0613     0.0274   missing   -625.1237    0.9989
    alpha[3]    0.5227    0.3104     0.1388   missing      1.4906    3.0788

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.1247    0.1444    0.4710    0.5658    0.9076
    alpha[2]    0.0000    0.0000    0.0000    0.0003    0.1235
    alpha[3]    0.0924    0.4339    0.5290    0.7404    0.8440


opponent_action=3 my_action=2

Chains MCMC chain (5×5×1 Array{Float64,3}):

Log evidence      = -2.5060689865157935
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Missing    Float64   Float64 

    alpha[1]    0.5647    0.3312     0.1481   missing     1.3986    4.6286
    alpha[2]    0.3823    0.2680     0.1199   missing     1.3958    4.7266
    alpha[3]    0.0530    0.0983     0.0440   missing   -49.0257    0.9867

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.1936    0.3928    0.4491    0.8357    0.9605
    alpha[2]    0.0386    0.1643    0.5509    0.5697    0.5988
    alpha[3]    0.0000    0.0000    0.0010    0.0374    0.2076


in simulation 10 first player chose 3 second player chose 1


20-element Array{Any,1}:
 (1, 2)
 (2, 1)
 (1, 2)
 (2, 3)
 (1, 1)
 (2, 3)
 (1, 1)
 (2, 2)
 (1, 3)
 (2, 1)
 (1, 2)
 (2, 1)
 (1, 2)
 (2, 3)
 (1, 1)
 (2, 3)
 (1, 1)
 (2, 2)
 (1, 3)
 (2, 1)